# Advent of Code 2025 - Day 10

In [1]:
# Test data
import re

test = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""

test_lines = test.splitlines()

test_square_rows = []
test_parens_rows = []
test_curly_rows = []

for line in test_lines:
    test_square_rows.append(re.findall(r'\[(.*?)\]', line)[0])

    parens = []
    for grp in re.findall(r'\((.*?)\)', line):
        nums = [int(x) for x in grp.split(',')]
        parens.append(nums)
    test_parens_rows.append(parens)

    curly_grp = re.findall(r'\{(.*?)\}', line)[0]
    test_curly_rows.append([int(x) for x in curly_grp.split(',')])

print(f"Test rows: {len(test_square_rows)}")


Test rows: 3


## Part 1

In [2]:
from collections import deque

def find_min_buttons(target, buttons):
    """
    Find minimum button presses to match target pattern.
    Each button flips indices (toggles . to # and vice versa).
    Uses BFS to find shortest path.
    """
    n = len(target)
    initial = '.' * n

    if initial == target:
        return 0

    # BFS: (state, steps)
    queue = deque([(initial, 0)])
    visited = {initial}

    while queue:
        current_state, steps = queue.popleft()

        for button in buttons:
            # Flip indices
            new_state = list(current_state)
            for idx in button:
                if idx < len(new_state):
                    new_state[idx] = '#' if new_state[idx] == '.' else '.'
            new_state = ''.join(new_state)

            if new_state == target:
                return steps + 1

            if new_state not in visited:
                visited.add(new_state)
                queue.append((new_state, steps + 1))

    return None

# Test example
results = []
for target, buttons in zip(test_square_rows, test_parens_rows):
    min_count = find_min_buttons(target, buttons)
    results.append(min_count)

print(f"Test results: {results}")
print(f"Sum: {sum(results)}")


Test results: [2, 3, 2]
Sum: 7


In [3]:
# Load real input
with open('input2.txt', 'r') as file:
    lines = file.read().strip().splitlines()

square_rows = []
parens_rows = []
curly_rows = []

for line in lines:
    square_rows.append(re.findall(r'\[(.*?)\]', line)[0])

    parens = []
    for grp in re.findall(r'\((.*?)\)', line):
        nums = [int(x) for x in grp.split(',')]
        parens.append(nums)
    parens_rows.append(parens)

    curly_grp = re.findall(r'\{(.*?)\}', line)[0]
    curly_rows.append([int(x) for x in curly_grp.split(',')])

print(f"Input rows: {len(square_rows)}")

Input rows: 162


In [4]:
# Part 1 - Real input
results = []
for target, buttons in zip(square_rows, parens_rows):
    min_count = find_min_buttons(target, buttons)
    results.append(min_count)

print(f"Part 1 Sum: {sum(results)}")
sum(results)

Part 1 Sum: 449


449

## Part 2

In [5]:
import numpy as np
from scipy.optimize import linprog

def find_min_buttons_part2(target, buttons):
    """
    Find minimum button presses to reach target values.
    Each button press increments values at specified indices by 1.
    Uses integer linear programming to solve efficiently.
    """
    goal = np.array(target)
    moves = [set(button) for button in buttons]

    # Objective: minimize total button presses (all coefficients = 1)
    c = np.ones(len(moves))

    # Constraints: for each index i, sum of button presses affecting i = goal[i]
    A_eq = []
    b_eq = []
    for i in range(len(goal)):
        A_eq.append([1 if i in move else 0 for move in moves])
        b_eq.append(goal[i])

    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)

    # Solve: minimize c^T * x subject to A_eq * x = b_eq, x >= 0, x integer
    res = linprog(
        c, A_eq=A_eq, b_eq=b_eq, bounds=(0, None),
        method="highs", integrality=True
    )

    return res.fun if res.success else None

# Test example
results = []
for target, buttons in zip(test_curly_rows, test_parens_rows):
    min_count = find_min_buttons_part2(target, buttons)
    results.append(min_count)

print(f"Test results: {results}")
print(f"Sum: {sum(results)}")

Test results: [10.0, 12.0, 11.0]
Sum: 33.0


In [6]:
# Part 2 - Real input
results = []
for target, buttons in zip(curly_rows, parens_rows):
    min_count = find_min_buttons_part2(target, buttons)
    results.append(min_count)

print(f"Part 2 Sum: {sum(results)}")
sum(results)

Part 2 Sum: 17848.0


17848.0